In [1]:
import numpy as np
import math
from tqdm.notebook import tqdm
from functools import cache
import copy

import heapq

import matplotlib.pyplot as plt


In [2]:
with open("input_day_23.txt") as f:
    text = f.read()


test = False
if test:
    text = r"""
#.#####################
#.......#########...###
#######.#########.#.###
###.....#.>.>.###.#.###
###v#####.#v#.###.#.###
###.>...#.#.#.....#...#
###v###.#.#.#########.#
###...#.#.#.......#...#
#####.#.#.#######.#.###
#.....#.#.#.......#...#
#.#####.#.#.#########v#
#.#...#...#...###...>.#
#.#.#v#######v###.###v#
#...#.>.#...>.>.#.###.#
#####v#.#.###v#.#.###.#
#.....#...#...#.#.#...#
#.#########.###.#.#.###
#...###...#...#...#.###
###.###.#.###v#####v###
#...#...#.#.>.>.#.>.###
#.###.###.#.###.#.#v###
#.....###...###...#...#
#####################.#
"""

text = text.strip()


In [3]:
G_strs = text.split("\n")
R = len(G_strs)
C = len(G_strs[0])

G = []
for r_str in G_strs:
    G.append([])
    for c in range(len(r_str)):
        G[-1].append(r_str[c])

print(R, C)


141 141


In [4]:
num_states_considered = 0
longest_dist = -1

min_curr_acc_heat_losses = {}

q = []
heapq.heapify(q)

# pos, visited_poses
first_state = ((0,1), set((0,1)))

heapq.heappush(q, first_state)

while len(q) > 0:

    current_state = heapq.heappop(q)

    curr_pos, curr_visited_poses = current_state
    r, c = curr_pos
    
    num_states_considered += 1

    if num_states_considered % 10000 == 0:
        print(f"    {num_states_considered} states considered, longest_dist: {longest_dist-2}")

    if current_state[0] == (R-1, C-2):
        longest_dist = max(longest_dist, len(curr_visited_poses))
        continue

    for next_dir in [(-1, 0), (1, 0), (0, -1), (0, 1)]:

        if G[r][c] == "^" and next_dir != (-1,0):
            continue
        if G[r][c] == ">" and next_dir != (0,1):
            continue
        if G[r][c] == "<" and next_dir != (0,-1):
            continue
        if G[r][c] == "v" and next_dir != (1,0):
            continue

        next_pos = (r + next_dir[0], c + next_dir[1])

        if G[next_pos[0]][next_pos[1]] == "#":
            continue

        if next_pos in curr_visited_poses:
            continue
    
        next_visited_poses = set(curr_visited_poses)
        next_visited_poses.add(next_pos)

        next_state = (next_pos, next_visited_poses)
        heapq.heappush(q, next_state)

print("num states checked:", num_states_considered)

print(f"part 1:    all states exhausted, longest_dist = {longest_dist-2}")

    10000 states considered, longest_dist: -3
    20000 states considered, longest_dist: -3
    30000 states considered, longest_dist: -3
    40000 states considered, longest_dist: -3
    50000 states considered, longest_dist: -3
    60000 states considered, longest_dist: -3
    70000 states considered, longest_dist: -3
    80000 states considered, longest_dist: -3
    90000 states considered, longest_dist: -3
    100000 states considered, longest_dist: -3
    110000 states considered, longest_dist: -3
    120000 states considered, longest_dist: -3
    130000 states considered, longest_dist: -3
    140000 states considered, longest_dist: -3
    150000 states considered, longest_dist: -3
    160000 states considered, longest_dist: -3
num states checked: 162037
part 1:    all states exhausted, longest_dist = 2386


In [5]:
intersection_points = set()
intersection_points.add((0,1))
intersection_points.add((R-1,C-2))

teleportations = {}

for r in range(R):
    for c in range(C):
        if G[r][c] == "#":
            continue
        num_open = 0
        for d in [(1,0), (-1,0), (0,1), (0,-1)]:
            if not (0 <= r+d[0] < R and 0 <= c+d[1] < C):
                continue
            if (   G[r+d[0]][c+d[1]] == "."
                or G[r+d[0]][c+d[1]] == "<"
                or G[r+d[0]][c+d[1]] == ">"
                or G[r+d[0]][c+d[1]] == "^"
                or G[r+d[0]][c+d[1]] == "v"):
                    num_open += 1
        if num_open > 2:
            #print("adding")
            intersection_points.add((r, c))
        #print(r,c,num_open)
        

intersection_points

{(0, 1),
 (5, 7),
 (5, 59),
 (7, 89),
 (11, 109),
 (15, 41),
 (29, 13),
 (29, 107),
 (33, 63),
 (35, 33),
 (35, 81),
 (41, 137),
 (55, 57),
 (55, 103),
 (57, 17),
 (57, 137),
 (59, 31),
 (59, 75),
 (79, 109),
 (79, 137),
 (83, 57),
 (83, 87),
 (87, 11),
 (89, 43),
 (101, 41),
 (103, 127),
 (105, 87),
 (105, 99),
 (107, 53),
 (111, 17),
 (123, 113),
 (125, 63),
 (125, 83),
 (131, 137),
 (133, 39),
 (140, 139)}

In [6]:
def count_adjacent_intersections(pos):
    #print("counting adjacent intersections", pos)
    num_adjacent_intersections = 0
    for rr, cc in [(1,0), (-1,0), (0,1), (0,-1)]:
        if (pos[0]+rr, pos[1]+cc) in intersection_points:
            num_adjacent_intersections += 1
            #print("found one at", (pos[0]+rr, pos[1]+cc))
    return num_adjacent_intersections


In [7]:
teleportations = {}
for r in range(R):
    for c in range(C):
        #if (r,c) != (13,6):
        #    continue

        if (count_adjacent_intersections((r,c)) == 1 and 
            (G[r][c] == "." or G[r][c] == "<" or G[r][c] == ">" or G[r][c] == "^" or G[r][c] == "v")):

            #print("starting path finding", r, c)


            num_steps = 0

            curr_teleport_start = (r, c)
            curr_r, curr_c = r, c
            curr_G = copy.deepcopy(G)
            curr_G[curr_r][curr_c] = "#"
            while True:
                #print("in loop", curr_r, curr_c)
                for rr, cc in [(1,0), (-1,0), (0,1), (0,-1)]:
                    if not (0 <= curr_r + rr < R and 0 <= curr_c + cc < C):
                        #print("out of range, continue")
                        continue
                    if (curr_r+rr, curr_c+cc) in intersection_points:
                        #print("looking back toward intersection point, continue")
                        continue
                    if (    curr_G[curr_r+rr][curr_c+cc] == "."
                            or curr_G[curr_r+rr][curr_c+cc] == ">"
                            or curr_G[curr_r+rr][curr_c+cc] == "<"
                            or curr_G[curr_r+rr][curr_c+cc] == "^"
                            or curr_G[curr_r+rr][curr_c+cc] == "v"):

                        almost_end = (curr_r, curr_c)
                        curr_r, curr_c = curr_r + rr, curr_c + cc
                        curr_G[curr_r][curr_c] = "#"
                        num_steps += 1
                        #print("stepped", rr, cc, "to", curr_r, curr_c)
                        break
                    #else:
                    #    print("hit wall at", curr_r+rr, curr_c+cc, curr_G[curr_r+rr][curr_c+cc])

                if count_adjacent_intersections((curr_r, curr_c)) == 1 and (curr_r, curr_c) != (r, c):
                    #print("done finding path", curr_r, curr_c)
                    break
                    
            curr_teleport_end = (curr_r, curr_c)
            teleportations[curr_teleport_start] = (curr_teleport_end, almost_end, num_steps)
        
for k,v in teleportations.items():
    print(k, ":", v[0], v[1], v[2])


(1, 1) : (4, 7) (3, 7) 17
(4, 7) : (1, 1) (1, 2) 17
(5, 8) : (15, 40) (15, 39) 302
(5, 58) : (15, 42) (15, 43) 150
(5, 60) : (7, 88) (7, 87) 134
(6, 7) : (28, 13) (27, 13) 152
(6, 59) : (32, 63) (31, 63) 154
(7, 88) : (5, 60) (5, 61) 134
(7, 90) : (11, 108) (11, 107) 126
(8, 89) : (34, 81) (33, 81) 126
(11, 108) : (7, 90) (7, 91) 126
(11, 110) : (40, 137) (39, 137) 484
(12, 109) : (28, 107) (27, 107) 66
(15, 40) : (5, 8) (5, 9) 302
(15, 42) : (5, 58) (5, 57) 150
(16, 41) : (34, 33) (33, 33) 86
(28, 13) : (6, 7) (7, 7) 152
(28, 107) : (12, 109) (13, 109) 66
(29, 14) : (35, 32) (35, 31) 60
(29, 106) : (35, 82) (35, 83) 142
(29, 108) : (41, 136) (41, 135) 220
(30, 13) : (56, 17) (55, 17) 274
(30, 107) : (54, 103) (53, 103) 168
(32, 63) : (6, 59) (7, 59) 154
(33, 62) : (35, 34) (35, 35) 202
(33, 64) : (35, 80) (35, 79) 102
(34, 33) : (16, 41) (17, 41) 86
(34, 63) : (54, 57) (53, 57) 122
(34, 81) : (8, 89) (9, 89) 126
(35, 32) : (29, 14) (29, 15) 60
(35, 34) : (33, 62) (33, 61) 202
(35, 80)

In [8]:
num_states_considered = 0
longest_dist = -1

min_curr_acc_heat_losses = {}

q = []
heapq.heapify(q)

# pos, visited_poses
first_visited_poses = set()
first_visited_poses.add((0, 1))
first_state = ((0,1), first_visited_poses, 0)
#first_state = ((0,1), [(0,1)])

heapq.heappush(q, first_state)

use_teleport = True

while len(q) > 0:

    current_state = heapq.heappop(q)

    curr_pos, curr_visited_poses, curr_extra_steps = current_state
    r, c = curr_pos

    #print(current_state)

    #print("curr_visited_poses:", curr_visited_poses)
    
    num_states_considered += 1

    if num_states_considered % 1000000 == 0:
        print(f"    {num_states_considered} states considered, longest_dist: {longest_dist}")

    if curr_pos == (R-1, C-2):
        longest_dist = max(longest_dist, len(curr_visited_poses) + curr_extra_steps)
        continue

    # seems like there aren't any dead ends, so this is useless
    #possible_poses = [(r-1, c), (r+1, c), (r, c-1), (r, c+1)]
    #barriers = [G_with_dead_ends_closed[pos[0]][pos[1]] for pos in possible_poses]
    #if possible_poses.count("#") == 3: # dead end
    #    G_with_dead_ends_closed[r][c] = "#"
    #    print("closing dead end")
    #    continue

    teleported = False
    if curr_pos in teleportations:
        next_pos, almost_end, additional_steps = teleportations[curr_pos]
        if not (next_pos in curr_visited_poses):
            next_visited_poses = set(curr_visited_poses)
            next_visited_poses.add(curr_pos)
            next_visited_poses.add(almost_end)
            next_visited_poses.add(next_pos)
            #print("teleporting from", curr_pos, "to", next_pos)
            #print("next_visited_poses:", next_visited_poses)


            next_extra_steps = curr_extra_steps + additional_steps - 2

            next_state = (next_pos, next_visited_poses, next_extra_steps)

            heapq.heappush(q, next_state)
            teleported = True
            
    if not teleported:
        for next_dir in [(-1, 0), (1, 0), (0, -1), (0, 1)]:

            next_pos = (r + next_dir[0], c + next_dir[1])

            if G[next_pos[0]][next_pos[1]] == "#":
                continue

            if next_pos in curr_visited_poses:
                continue
        
            # seems like using an array isn't faster than using a set
            #next_visited_poses = curr_visited_poses + [next_pos]

            next_visited_poses = set(curr_visited_poses)
            next_visited_poses.add(next_pos)

            next_state = (next_pos, next_visited_poses, curr_extra_steps)
            heapq.heappush(q, next_state)

print("num states checked:", num_states_considered)

print(f"part 2:    all states exhausted, longest_dist = {longest_dist-1}")


    1000000 states considered, longest_dist: -1
    2000000 states considered, longest_dist: -1
    3000000 states considered, longest_dist: -1
    4000000 states considered, longest_dist: -1
    5000000 states considered, longest_dist: -1
    6000000 states considered, longest_dist: -1
    7000000 states considered, longest_dist: -1
    8000000 states considered, longest_dist: -1
    9000000 states considered, longest_dist: -1
    10000000 states considered, longest_dist: -1
    11000000 states considered, longest_dist: -1
    12000000 states considered, longest_dist: -1
    13000000 states considered, longest_dist: -1
    14000000 states considered, longest_dist: -1
    15000000 states considered, longest_dist: -1
    16000000 states considered, longest_dist: -1
    17000000 states considered, longest_dist: -1
    18000000 states considered, longest_dist: -1
    19000000 states considered, longest_dist: -1
    20000000 states considered, longest_dist: -1
    21000000 states considere

In [9]:
"""
#.#####################
#.......#########...###
#######.#########.#.###
###.....#.>.>.###.#.###
###v#####.#v#.###.#.###
###.>...#.#.#.....#...#
###v###.#.#.#########.#
###...#.#.#.......#...#
#####.#.#.#######.#.###
#.....#.#.#.......#...#
#.#####.#.#.#########v#
#.#...#...#...###...>.#
#.#.#v#######v###.###v#
#...#.>.#...>.>.#.###.#
#####v#.#.###v#.#.###.#
#.....#...#...#.#.#...#
#.#########.###.#.#.###
#...###...#...#...#.###
###.###.#.###v#####v###
#...#...#.#.>.>.#.>.###
#.###.###.#.###.#.#v###
#.....###...###...#...#
#####################.#
"""

'\n#.#####################\n#.......#########...###\n#######.#########.#.###\n###.....#.>.>.###.#.###\n###v#####.#v#.###.#.###\n###.>...#.#.#.....#...#\n###v###.#.#.#########.#\n###...#.#.#.......#...#\n#####.#.#.#######.#.###\n#.....#.#.#.......#...#\n#.#####.#.#.#########v#\n#.#...#...#...###...>.#\n#.#.#v#######v###.###v#\n#...#.>.#...>.>.#.###.#\n#####v#.#.###v#.#.###.#\n#.....#...#...#.#.#...#\n#.#########.###.#.#.###\n#...###...#...#...#.###\n###.###.#.###v#####v###\n#...#...#.#.>.>.#.>.###\n#.###.###.#.###.#.#v###\n#.....###...###...#...#\n#####################.#\n'

In [10]:
"""num_states_considered = 0
longest_dist = -1

min_curr_acc_heat_losses = {}

q = []
heapq.heapify(q)

# pos, visited_poses
first_state = ((0,1), set((0,1)))
#first_state = ((0,1), [(0,1)])

heapq.heappush(q, first_state)

while len(q) > 0:

    current_state = heapq.heappop(q)

    curr_pos, curr_visited_poses = current_state
    r, c = curr_pos
    
    num_states_considered += 1

    if num_states_considered % 1000000 == 0:
        print(f"    {num_states_considered} states considered, longest_dist: {longest_dist-2}")

    if curr_pos == (R-1, C-2):
        longest_dist = max(longest_dist, len(curr_visited_poses))
        continue

    # seems like there aren't any dead ends, so this is useless
    #possible_poses = [(r-1, c), (r+1, c), (r, c-1), (r, c+1)]
    #barriers = [G_with_dead_ends_closed[pos[0]][pos[1]] for pos in possible_poses]
    #if possible_poses.count("#") == 3: # dead end
    #    G_with_dead_ends_closed[r][c] = "#"
    #    print("closing dead end")
    #    continue

    for next_dir in [(-1, 0), (1, 0), (0, -1), (0, 1)]:

        next_pos = (r + next_dir[0], c + next_dir[1])

        if G_with_dead_ends_closed[next_pos[0]][next_pos[1]] == "#":
            continue

        if next_pos in curr_visited_poses:
            continue
    
        # seems like using an array isn't faster than using a set
        #next_visited_poses = curr_visited_poses + [next_pos]

        next_visited_poses = set(curr_visited_poses)
        next_visited_poses.add(next_pos)

        next_state = (next_pos, next_visited_poses)
        heapq.heappush(q, next_state)

print(f"part 2:    all states exhausted, longest_dist = {longest_dist-2}")

"""

'num_states_considered = 0\nlongest_dist = -1\n\nmin_curr_acc_heat_losses = {}\n\nq = []\nheapq.heapify(q)\n\n# pos, visited_poses\nfirst_state = ((0,1), set((0,1)))\n#first_state = ((0,1), [(0,1)])\n\nheapq.heappush(q, first_state)\n\nwhile len(q) > 0:\n\n    current_state = heapq.heappop(q)\n\n    curr_pos, curr_visited_poses = current_state\n    r, c = curr_pos\n    \n    num_states_considered += 1\n\n    if num_states_considered % 1000000 == 0:\n        print(f"    {num_states_considered} states considered, longest_dist: {longest_dist-2}")\n\n    if curr_pos == (R-1, C-2):\n        longest_dist = max(longest_dist, len(curr_visited_poses))\n        continue\n\n    # seems like there aren\'t any dead ends, so this is useless\n    #possible_poses = [(r-1, c), (r+1, c), (r, c-1), (r, c+1)]\n    #barriers = [G_with_dead_ends_closed[pos[0]][pos[1]] for pos in possible_poses]\n    #if possible_poses.count("#") == 3: # dead end\n    #    G_with_dead_ends_closed[r][c] = "#"\n    #    print(

In [11]:
# 5 million in 2 minutes